# GT Introduction to Analytics Modeling - Week 7 HW

As directed in the assignment, we'll use the Pulp library to create a linear program to optimize the diet.  We'll also use other data manipulation libraries to help load the spreadsheet and prepare the data.  Note that some manual manipulation of the spreadsheet was performed outside of this notebook.  For example, a numeric column was created for serving size.

We'll first load the libraries we'll be using.

In [2]:
import pulp as pp
import pandas as pd

We'll use pandas to load our spreadsheet.  Note that I manually saved it as a CSV file, changed the Serving Size column to be numeric, and removed special characters from the food names and column names.

In [24]:
data = pd.read_csv("diet.csv", index_col=0)
data.head()

,Price Per Serving,Serving Size,Calories,Cholesterol mg,Total Fat g,Sodium mg,Carbohydrates g,Dietary Fiber g,Protein g,Vitamin A IU,Vitamin C IU,Calcium mg,Iron mg
Foods,,,,,,,,,,,,,
Frozen Broccoli,0.16,10.0,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
Carrots Raw,0.07,0.5,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
Celery Raw,0.04,1.0,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
Frozen Corn,0.18,0.5,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
Lettuce Iceberg Raw,0.02,1.0,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


### Constraints
Our constraints are as follows:
1. Minimum daily intake values 1500 30 20 800 130 125 60 1000 400 700 10.  We'll use the variable $x_j$ to represent the lower bound for attribute $j$.
2. Maximum daily intake values 2500 240 70 2000 450 250 100 10000 5000 1500 40. We'll use the variable $y_j$ to represent the upper bound for attribute $j$.
3. If a food is selected, then a minimum of 1/10 serving must be chosen.
4. Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.
5. To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected.

### Variables
We'll need a binary variable for each food to indicate whether or not we select that food in our model.  The variable $f_i = 1$ indicates that we selected food $i$. We'll also need a numeric variable for each food to indicate the serving size.  The variable $s_i$ reprsents the serving size of food $i$.  These will be used to solve the constraints as follows:

1. The cumulative sum of the product of each food's binary variable, the serving size and nutrition attribute (e.g calories) must be greater than or equal to the corresponding nutritional constraint.  This gives the equation $\sum f_i \times s_i \times a_{ij} \ge x_j$ where $a_{ij}$ represents nutritional attribute $j$ for food $i$, and $x_j$ represents the lower bound nutritional constraint for attribute $j$.
2. The cumulative sum of the product of each food's binary variable, the serving size and nutrition attribute (e.g calories) must be less than or equal to the corresponding nutritional constraint.  This gives the equation $\sum f_i \times s_i \times a_{ij} \le y_j$ where $a_{ij}$ represents nutritional attribute $j$ for food $i$, and $y_j$ represents the upper bound nutritional constraint for attribute $j$.
3. The variable representing the serving size of a food $s_i$ must be greather than 1/10 * the binary variable $f_i$ for the food. This constraint yields the equation $s_i \ge .1 \times f_i$.
4. The sum of the binary variable for celery and frozen broccoli must be less than or equal to 1. This constraint yields the equation $x_a + x_b \le 1$ where $a, b \in (celery, frozen broccoli)$.
5. The sum of the binary variables that correspond to meat/poultry/fish/eggs must be greater than or equal to 3. This constraint yields the equation $\sum x_a \ge 3$ where $a \in (meat/poultry/fish/eggs)$.

### Objective Function
The objective function is to minimize the cumulative sum of the product of each food's binary variable, serving size, and cost. This yields the following equation which we will minimize.

$\sum f_i \times s_i \times c_i$

### Data Setup
Pulp uses dictionaries to represent the data to be applied to the constraints.  The following code snippets extract the dictionaries from our pandas dataframe.  We extract the column names for all relavent data, and we also create a list of names the specfically represent nutritional values.  With the list of names, we can create all of the necessary dictionaries.

In [57]:
#our food names
foods = data.index.tolist()
print("Foods: ", foods)
print()

#our column names
names = data.columns.tolist()

#extract nutrition names
nutrition = [i for i in names if i not in ['Price Per Serving','Serving Size']]

print("Column Names: ", names)
print()
print("Nutrition Names", nutrition) 

#create all of the dictionaries we'll need
dataDict = {k:data[k].to_dict() for k in names}
#print(dataDict)

#Minimum daily intake constraints 1500 30 20 800 130 125 60 1000 400 700 10
minvalues = [1500.0, 30.0, 20.0, 800.0, 130.0, 125.0, 60.0, 1000.0, 400.0, 700.0, 10.0]
minNutrition = {k:v for k,v in zip(nutrition, minvalues)}

#Maximum daily intake constraints 2500 240 70 2000 450 250 100 10000 5000 1500 40
maxvalues = [2500.0, 240.0, 70.0, 2000.0, 450.0, 250.0, 100.0, 10000.0, 5000.0, 1500.0, 40.0]
maxNutrition = {k:v for k,v in zip(nutrition, maxvalues)}

print()
print("Maximum Intakes", maxNutrition)
print()
print("Minimum Intakes", minNutrition)

protiens = ['Roasted Chicken', 'Poached Eggs', 'Scrambled Eggs', 'Bologna Turkey', 'Frankfurter Beef', 
            'Ham Sliced Extralean', 'Kielbasa Prk', 'Taco', 'Hamburger W Toppings', 'Hotdog Plain', 'Pork', 
            'Sardines in Oil', 'White Tuna in Water', 'Chicknoodl Soup', 'Vegetbeef Soup', 'Neweng Clamchwd', 
            'New E Clamchwd']

print()
print("Protiens: ", protiens)

Foods:  ['Frozen Broccoli', 'Carrots Raw', 'Celery Raw', 'Frozen Corn', 'Lettuce Iceberg Raw', 'Peppers Sweet Raw', 'Potatoes Baked', 'Tofu', 'Roasted Chicken', 'Spaghetti W Sauce', 'Tomato Red Ripe Raw', 'Apple Raw W Skin', 'Banana', 'Grapes', 'Kiwifruit Raw Fresh', 'Oranges', 'Bagels', 'Wheat Bread', 'White Bread', 'Oatmeal Cookies', 'Apple Pie', 'Chocolate Chip Cookies', 'Butter Regular', 'Cheddar Cheese', 'Fat Whole Milk', 'Lowfat Milk', 'Skim Milk', 'Poached Eggs', 'Scrambled Eggs', 'Bologna Turkey', 'Frankfurter Beef', 'Ham Sliced Extralean', 'Kielbasa Prk', 'Cap Crunch', 'Cheerios', 'Corn Flks Kelloggs', 'Raisin Bran Kelloggs', 'Rice Krispies', 'Special K', 'Oatmeal', 'Malt-O-Meal Choc', 'Pizza W Pepperoni', 'Taco', 'Hamburger W Toppings', 'Hotdog Plain', 'Couscous', 'White Rice', 'Macaroni Ckd', 'Peanut Butter', 'Pork', 'Sardines in Oil', 'White Tuna in Water', 'Popcorn Air-Popped', 'Potato Chips Bbqflvr', 'Pretzels', 'Tortilla Chip', 'Chicknoodl Soup', 'Splt Pea Hamsoup', 'Veg

### Pulp Model Setup
Now that we have our relevant data dictionaries, we can setup our Pulp model and the constraints.


In [64]:
model = pp.LpProblem("The Diet Problem", pp.LpMinimize)

# setup our decision variables
f_vars = pp.LpVariable.dicts("f", foods, 0, 1, pp.LpInteger)
s_vars = pp.LpVariable.dicts("s", foods, 0, None, pp.LpContinuous)


# add the objective function to the model
model += pp.lpSum([dataDict['Price Per Serving'][i]*f_vars[i] for i in foods]), "Total Cost Diet"

#Constraint 1 - minimum intake
for n in nutrition:
    model += pp.lpSum([dataDict[n][i] * f_vars[i]  for i in foods]) >= minNutrition[n], "Min"+n+"Req"

#Constraint 2 - maximum intake
for n in nutrition:
    model += pp.lpSum([dataDict[n][i] * f_vars[i] for i in foods]) <= maxNutrition[n], "Max"+n+"Req"

#Constraint 3 - minimum serving size, if selected
#model += pp.lpSum([dataDict['Serving Size'][i]*f_vars[i] for i in foods]) >= .1, "Total Cost Diet"

#Constraint 4 - not both broccoli and celery
model += pp.lpSum([f_vars[i] for i in ['Celery Raw', 'Frozen Broccoli']]) <= 1, "Broccoli or Celery Req"

#Constraint 5 - Mimimum 3 protiens
model += pp.lpSum([f_vars[i] for i in protiens]) >= 3, "Protiens Req"


In [65]:
print(model)

The Diet Problem:
MINIMIZE
0.16*f_Apple_Pie + 0.24*f_Apple_Raw_W_Skin + 0.16*f_Bagels + 0.15*f_Banana + 0.67*f_Beanbacn_Soup + 0.15*f_Bologna_Turkey + 0.05*f_Butter_Regular + 0.31*f_Cap_Crunch + 0.07*f_Carrots_Raw + 0.04*f_Celery_Raw + 0.25*f_Cheddar_Cheese + 0.28*f_Cheerios + 0.39*f_Chicknoodl_Soup + 0.03*f_Chocolate_Chip_Cookies + 0.28*f_Corn_Flks_Kelloggs + 0.39*f_Couscous + 0.65*f_Crm_Mshrm_Soup + 0.16*f_Fat_Whole_Milk + 0.27*f_Frankfurter_Beef + 0.16*f_Frozen_Broccoli + 0.18*f_Frozen_Corn + 0.32*f_Grapes + 0.33*f_Ham_Sliced_Extralean + 0.83*f_Hamburger_W_Toppings + 0.31*f_Hotdog_Plain + 0.15*f_Kielbasa_Prk + 0.49*f_Kiwifruit_Raw_Fresh + 0.02*f_Lettuce_Iceberg_Raw + 0.23*f_Lowfat_Milk + 0.17*f_Macaroni_Ckd + 0.52*f_Malt_O_Meal_Choc + 0.99*f_New_E_Clamchwd + 0.75*f_Neweng_Clamchwd + 0.82*f_Oatmeal + 0.09*f_Oatmeal_Cookies + 0.15*f_Oranges + 0.07*f_Peanut_Butter + 0.53*f_Peppers_Sweet_Raw + 0.44*f_Pizza_W_Pepperoni + 0.08*f_Poached_Eggs + 0.04*f_Popcorn_Air_Popped + 0.81*f_Pork + 0.2

### Pulp Model Solution
With our object function defined and our constraints setup, we can now solve the modle and output the results.

In [ ]:
# The problem is solved using PuLP's choice of Solver
model.solve()

# The status of the solution is printed to the screen
print(pp.pStatus[model.status])

# Each of the variables is printed with it's resolved optimum value
for v in model.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen
print "Total Cost of Ingredients per can = ", pp.value(model.objective)